In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("my-experiment-2")

<Experiment: artifact_location='s3://mlflow-artifacts-remote-zoomcamp/1', creation_time=1688358175294, experiment_id='1', last_update_time=1688358175294, lifecycle_stage='active', name='my-experiment-2', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_csv(filename)

    # Rename all the column to be nice to see
    df['Trending'] = df.shape[0] + 1 - df['Views'].rank()
    df = df.sort_values(by='Trending', ascending=True)

    # Drop some unnecessary file
    df = df.dropna()
    df = df.drop('Url_spotify', axis=1)
    df = df.drop('Uri', axis=1)

    categorical = ['Licensed']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['Licensed']
    numerical = ['Views', 'Likes']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
df = read_dataframe('/home/ubuntu/mlops_project/data/spotify-and-youtube/Spotify_Youtube.csv')

split_number = int(len(df)/2)

df_train = df[:split_number]
df_val = df[split_number:]

target = 'Trending'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [7]:
from sklearn.linear_model import LinearRegression

with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)
    
    pipeline = make_pipeline(
    DictVectorizer(),
    LinearRegression()
    )
    
    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    
    range = df['Trending'].max() - df['Trending'].min()
    error_percentage = rmse / range * 100
    mlflow.log_metric("error_percentage", error_percentage)
    
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 9609.787951762808


/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
with mlflow.start_run() as run:
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)
    
    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )
    
    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    range = df['Trending'].max() - df['Trending'].min()
    error_percentage = rmse / range * 100
    mlflow.log_metric("error_percentage", error_percentage)

    with open("/tmp/dict_vectorizer.bin", "wb") as f_out:
        pickle.dump(pipeline[0], f_out)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")
    mlflow.log_artifact("/tmp/dict_vectorizer.bin", artifact_path="dict_vectorizer")
    
    run_id = run.info.run_id # get run id


In [ ]:
from mlflow.tracking import MlflowClient

In [ ]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '0baad4b2afeb41b48e1469722bd12fda'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer/dict_vectorizer.bin')

In [ ]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)